# Data engineer skills Project

The objective of this project is to highlight the most in-demand skills for data engineer roles in France.

How it is done : 
1. Extract job postings for data engineer roles from a personal database.
2. Format description and look for most demanded skills.
3. Summarize the results visually.

## Imports

In [160]:
import os
import pandas as pd
import logging
from datetime import datetime
from sqlalchemy import create_engine, Table, MetaData, text
import re
import regex as re

## Configuration

In [161]:
# Logging
LOG_DIR = "logs"
os.makedirs(LOG_DIR, exist_ok=True)
logging.basicConfig(
    filename=os.path.join(LOG_DIR, f"pipeline_{datetime.now().strftime('%Y-%m-%d')}.log"),
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

In [162]:
##################  VARIABLES  ##################
# Param Database PostgreSQL
DB_NAME = os.environ.get("DB_NAME", "jobsdb")
DB_USER = os.environ.get("DB_USER","jobsuser")
DB_PASS = os.environ.get("DB_PASS", "jobspass")
DB_HOST = os.environ.get("DB_HOST", "localhost")
DB_PORT = os.environ.get("DB_PORT","5432")

# Nom table
DB_TABLE_NAME = "offres_table"

## Extract data from database

In [163]:
### Connect to database
def export_from_database(engine, table_name):
    # logging.info("Connect to database.")
    try:  
        query = text(f"""
            SELECT     
                   id, 
                   source,
                   recherche,
                   titre,
                   description,
                   departement,
                   date_publication               
            FROM {table_name}
            WHERE recherche = 'data engineer'
        """)
        
        with engine.connect() as conn:
            result = conn.execute(query)       
            df_extract = pd.DataFrame(result.fetchall(),columns=["id", 
                                                               "source",
                                                               "recherche",
                                                               "titre",
                                                               "description",
                                                               "departement",
                                                               "date_publication"])    
            return df_extract
            
    except requests.RequestException as e:
        logging.error(f"Error database export: {e}")
        return []

In [164]:
# Connexion DB
engine = create_engine(f"postgresql+psycopg2://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

#Extract data from DB
df = export_from_database(engine, DB_TABLE_NAME)
# df = df.head(50)
display(df)

,id,source,recherche,titre,description,departement,date_publication
0,202FHJH,France Travail,data engineer,Knowledge Manager IA (H/F),CDI - Consultant Knowledge Manager IA (H/F)\n\...,91,2026-01-06
1,200MWYF,France Travail,data engineer,Ingénieur Gestion des Données Techniques (H/F),Rattaché au Responsable de Groupe Nouveaux Pro...,68,2025-11-19
2,7407914,France Travail,data engineer,Senior Consultant/Manager – Data Architect et/...,Description de l'entrepriseRejoignez un cabine...,92,2026-01-05
3,4882600908,Adzuna,data engineer,Data Engineer (H/F),Missions : Mise en place et la configuration d...,47,2024-10-01
4,8033210,France Travail,data engineer,Chef de projet DATA H/F,Mission principale : piloter la conception et ...,34,2026-01-19
...,...,...,...,...,...,...,...
4677,5558161529,Adzuna,data engineer,Lead Software Data Engineer – Data & AI Digita...,Wake IT UP recrute pour son client : client fi...,75,2025-12-28
4678,5395847606,Adzuna,data engineer,Data Engineer (Hadoop-Scala) Junior H/F (IT),Contexte et objectifs de la prestation : Le dé...,94,2025-09-12
4679,5558161326,Adzuna,data engineer,Data engineer Python GCP,Proxiad Ouest est à la recherche d’un nouveau ...,44,2025-12-28
4680,5504021111,Adzuna,data engineer,Data Engineer Python (IT),Mission : Data Engineer Python Localisation : ...,93,2025-11-19


## Clean and normalize job description

In [165]:
def clean_text(text):
    # logging.info("Clean and normalize job description.")
    try: 
        # Converts everything to lowercase
        text = text.lower()
    
        # It replaces anything like <tag>...</tag> with a space.
        text = re.sub(r"<.*?>", " ", text)     # HTML
    
        # Removes unwanted characters (but keeps French accents)
        # This line keeps only:
        # - lowercase letters a–z
        # - digits 0–9
        # - + . /
        # - spaces
        # Everything else becomes a space.
        text = re.sub(r"[^\p{L}0-9+./ ]", " ", text)
    
        # Cleans extra spaces
        # - \s+ = multiple spaces
        # - Replaces them with one single space
        # - .strip() removes spaces at the beginning and end
        text = re.sub(r"\s+", " ", text).strip()
        return text
    except requests.RequestException as e:
        logging.error(f"Error cleaning and normalizing process: {e}")
        return []

df["clean_description"] = df["description"].astype(str).apply(clean_text)

## Extract skills

In [166]:
DATA_ENGINEER_ONTOLOGY = {
    "Langages": [
        "python", "sql", "scala", "java", "bash"
    ],
    "Bases_de_donnees": [
        "postgresql", "mysql", "sql server", "oracle",
        "snowflake", "bigquery", "redshift", "synapse",
        "mongodb", "cassandra", "dynamodb", "elasticsearch"
    ],
    "Modelisation_et_architecture": [
        "data modeling", "dimensional modeling",
        "star schema", "snowflake schema",
        "data warehouse", "data lake", "lakehouse",
        "lambda architecture", "medallion architecture"
    ],
    "Pipelines_et_orchestration": [
        "etl", "elt", "data pipeline", "ingestion",
        "airflow", "dagster", "prefect"
    ],
    "Big_Data": [
        "spark", "pyspark", "hadoop",
        "distributed computing", "partitioning"
    ],
    "Streaming": [
        "kafka", "kinesis", "pubsub", "event streaming"
    ],
    "Cloud": [
        "aws", "gcp", "azure",
        "s3", "gcs", "adls",
        "glue", "dataproc", "databricks"
    ],
    "DevOps_DataOps": [
        "docker", "kubernetes", "ci/cd",
        "terraform", "infrastructure as code"
    ],
    "Qualite_et_gouvernance": [
        "data quality", "data validation", "monitoring",
        "data governance", "data lineage", "rgpd"
    ],
    "Analytics_enablement": [
        "dbt", "semantic layer",
        "power bi", "tableau", "looker"
    ]
}

In [167]:
def extract_skills(text, ontology):
    # logging.info("Extract skills from job description.")
    try:     
        found_skills = set()
        for category, skills in ontology.items():
            for skill in skills:
                if skill in text:
                    found_skills.add(skill)
        return list(found_skills)
    except requests.RequestException as e:
        logging.error(f"Error during extracting skills process: {e}")
        return []

df["skills_found"] = df["clean_description"].apply(lambda x: extract_skills(x, DATA_ENGINEER_ONTOLOGY))
df[["id", "skills_found"]].head()

,id,skills_found
0,202FHJH,[]
1,200MWYF,[]
2,7407914,"[s3, aws, data lake, gcp, azure, lakehouse]"
3,4882600908,[]
4,8033210,"[elt, data lake, etl, snowflake, dbt]"


In [168]:
# Long-form exploitation
skills_long = df.explode("skills_found")
skills_long = skills_long.dropna(subset=["skills_found"])

skills_long

,id,source,recherche,titre,description,departement,date_publication,clean_description,skills_found
2,7407914,France Travail,data engineer,Senior Consultant/Manager – Data Architect et/...,Description de l'entrepriseRejoignez un cabine...,92,2026-01-05,description de l entrepriserejoignez un cabine...,s3
2,7407914,France Travail,data engineer,Senior Consultant/Manager – Data Architect et/...,Description de l'entrepriseRejoignez un cabine...,92,2026-01-05,description de l entrepriserejoignez un cabine...,aws
2,7407914,France Travail,data engineer,Senior Consultant/Manager – Data Architect et/...,Description de l'entrepriseRejoignez un cabine...,92,2026-01-05,description de l entrepriserejoignez un cabine...,data lake
2,7407914,France Travail,data engineer,Senior Consultant/Manager – Data Architect et/...,Description de l'entrepriseRejoignez un cabine...,92,2026-01-05,description de l entrepriserejoignez un cabine...,gcp
2,7407914,France Travail,data engineer,Senior Consultant/Manager – Data Architect et/...,Description de l'entrepriseRejoignez un cabine...,92,2026-01-05,description de l entrepriserejoignez un cabine...,azure
...,...,...,...,...,...,...,...,...,...
4674,5512129731,Adzuna,data engineer,Data Engineer Migration Databricks H/F,"En tant que Data Engineer, nous vous proposons...",1,2025-11-25,en tant que data engineer nous vous proposons ...,hadoop
4674,5512129731,Adzuna,data engineer,Data Engineer Migration Databricks H/F,"En tant que Data Engineer, nous vous proposons...",1,2025-11-25,en tant que data engineer nous vous proposons ...,databricks
4674,5512129731,Adzuna,data engineer,Data Engineer Migration Databricks H/F,"En tant que Data Engineer, nous vous proposons...",1,2025-11-25,en tant que data engineer nous vous proposons ...,ci/cd
4675,5515849223,Adzuna,data engineer,Lead Data Engineer Snowflake H/F,Lead Data Engineer Snowflake - Construisez et ...,75,2025-11-27,lead data engineer snowflake construisez et fa...,snowflake


## Frequency statistics

In [170]:
total_offres = df["id"].nunique()

print(total_offres)

skills_stats = (
    skills_long.groupby("skills_found")["id"]
    .nunique()
    .reset_index()
    .rename(columns={"id": "nb_offres"})
)

skills_stats["pourcentage"] = (skills_stats["nb_offres"] / total_offres * 100).round(1)

skills_stats = skills_stats.sort_values("pourcentage", ascending=False)

skills_stats.head()

4682


,skills_found,nb_offres,pourcentage
55,sql,772,16.5
47,python,684,14.6
51,scala,553,11.8
3,azure,513,11.0
54,spark,509,10.9
